![HPEDEVlogo](Pictures/hpedevlogo-NB.JPG)    ![Dockerlogo](Pictures/docker-compose.png)  

In [ ]:
%login {{ DINDSRVNAME }}

# Using Docker compose

Docker compose is a tool part of the Docker ecosystem.
It is used to run solutions split in multiple containers which is the case most of the time.
This is mainly due to the Docker philosophy to use one container per service.

Another benefit is to define the container running parameters within a YAML configuration file.

## Installing Docker compose

If you're not in the Docker in Docker setup, Use the following commands:

In [ ]:
curl -L https://github.com/docker/compose/releases/download/1.9.0/docker-compose-$(uname -s)-$(uname -m) > /usr/local/bin/docker-compose

If that command fails and you're on MacOS X (because the File System is read-only, then use the target /opt/local/sbin/docker-compose instead)

In [ ]:
chmod +x /usr/local/bin/docker-compose

Check that the binary works by displaying the revision (providing you have /usr/local/bin in your path):

In [ ]:
docker-compose --version

## Our first docker-compose.yml file
Now we have a working docker-compose, we need to create an application environment and our first **docker-compose.yml** configuration file.

Create the build environment by moving all our previous stuffs into a folder:

In [ ]:
mkdir owncloud
mv Dockerfile owncloud-7.0.15.tar.bz2 config.php owncloud
cd owncloud

Now we can create our configuration file. We will use the new v3.0 format instead of the legacy one. The v3.0 was created to extend functionalities and can be activated by specifying the release at the top of the file.

Note : Of course old docker-compose binaries don't manage v3.0, you can find support information [here](https://docs.docker.com/compose/compose-file/#compose-and-docker-compatibility-matrix)

In [ ]:
cat > docker-compose.yml << EOF`
```
version: '3'
services:
  web:
    build: .
    volumes:
      - /data:/data
    ports:
      - "80:80"
EOF
```

In [ ]:
The above file asks to docker-compose to define a web service that will be built from our Dockerfile, to expose port 80 and to map /data on the host to /data in the container.

We can now start our application using:

In [ ]:
docker-compose up -d

In [ ]:
docker ps

Our application starts and should work the same way as previously. However it is much simpler because we don't need to define ports and storage mapping using the command line, also the YAML file can be held in and this information can be managed in Configuration Management System.

You can also note that the container name is defined as `application_service_number` (owncloud_web_1)

Now stop the application:

In [ ]:
docker-compose down

Check what happens to the container.

Ok that's cool, but it is not really a big change so far.

## Going further with docker-compose.yml

If we look at our owncloud application, we are using an internal sqlite database. This was defined during the setup phase.

As mentioned during the setup (below), this is convenient for a limited installation, but for larger ones it is better to use mysql/mariadb or postgres.

![Owncloud sqlite setup](Pictures/owncloud_setup.png)

In order to install owncloud on another database:

   1. Wipe `config.php` to have the setup page proposed again by the application.
   2. Add the `php-mysql` package to your Dockerfile in the relevant part.
   3. Start the application but use `docker-compose up -d --build` to force the rebuild of the Dockerfile.

![Owncloud sqlite setup](Pictures/owncloud_setup_db.png)

Instead of building our own mariadb container from scratch like we did for owncloud, we will use the official Docker one.

Of course it requires some information about the compose-file format. Documentation for this can be found here: https://docs.docker.com/compose/compose-file and the image itself there: https://hub.docker.com/_/mariadb

  1. Try to modify `docker-compose.yml` to add a db service based on the mariadb official images.
  2. We need to provide the database parameters fields (user, password etc...). Hint: Look at the mariadb container environment variables. **Discuss with your trainer if you're stuck or use our slack Channel!**
  3. What is the hostname of our container ? Hint: Look at the links or preferred network directive to allow db container connection from the web container.


If you didn't manage to configure the mariadb container and use it with owncloud, then the additional content for your docker-compose.yml could be useful:
```
  db:
    image: mariadb
    environment:
      - MYSQL_ROOT_PASSWORD=password
      - MYSQL_DATABASE=owncloud
      - MYSQL_USER=owncloud
      - MYSQL_PASSWORD=owncloudpwd
```

We are now using a mariadb container, but the database content is inside the container. So this is the same story as before, we need to keep our data persistent.

  1. Find out where are managed the db files.
  1. Use a Docker volume to use them from the host.
  2. Modify docker-compose.yml to do that. Hint: separate owncloud and db data under /data to avoid user rights conflicts.

If you manage to configure the mariadb container with persistent data your docker-compose.yml should look like this:
```
version: '3'
services:
  web:
    build: .
    volumes:
      - /data/owncloud:/data/owncloud
    ports:
      - "80:80"
    networks:
      - oclan
    depends_on:
      - db
  db:
    image: mariadb
    environment:
      - MYSQL_ROOT_PASSWORD=password
      - MYSQL_DATABASE=owncloud
      - MYSQL_USER=owncloud
      - MYSQL_PASSWORD=owncloudpwd
    networks:
      - oclan
    volumes:
      - /data/db:/var/lib/mysql
networks:
  oclan:
    driver: bridge
```

In [ ]:
docker-compose ps

Try to change the listening port inside your docker-compose.yml file and perform a `docker-compose -up -d`

You can notice that only the services that need to be modified are recreated.

You may like to try to allow scalability for your application by scaling the
web service

In [ ]:
docker-compose scale web=2

Detect whether this is working or not and why. If not, we'll find another way
to solve this.

Bonus, you can try to update the docker-compose.yml file to add an ha-proxy instance in front of the web services.

In [ ]:
# Using docker-machine to create Docker hosts

Depending on the context of the Lab, you may already have enough machines available (5) to run the Swarm part, or you may need to create them. In that case, continue with this part, if not, skip to the next one.

docker-machine is a Docker tool that can be used to deploy Docker hosts on various cloud platforms (AWS, Azure, Digital Ocean, Openstack, etc...).
We will use this tool to deploy 5 nodes that will be used later in the Swarm part. Docker machine simply deploys a server on your favorite provider and then installs the latest release of Docker Engine.

The following command will deploy one node to our openstack environment but will not run it yet.

In [ ]:
docker-machine create --driver openstack --openstack-auth-url http://10.11.50.26:5000/v2.0 --openstack-flavor-name m1.small --openstack-image-name ubuntu1604 --openstack-username dockerlab --openstack-password Linux1 --openstack-tenant-name dockerlab --openstack-net-name private --openstack-floatingip-pool external-network --openstack-sec-groups default --openstack-ssh-user ubuntu dockerw1

In [ ]:
In order to save time we will deploy 5 hosts in parallel with the following command.

In [ ]:
for i in dockerm1 dockerm2 dockerm3 dockerw1 dockerw2; do
    docker-machine create --driver openstack --openstack-auth-url http://10.11.50.26:5000/v2.0 --openstack-flavor-name m1.small --openstack-image-name ubuntu1604 --openstack-username dockerlab --openstack-password linux1 --openstack-tenant-name dockerlab --openstack-net-name private --openstack-floatingip-pool external-network --openstack-sec-groups default --openstack-ssh-user ubuntu $i &
done

In [ ]:
This will take around 5mn. You can list the machines installed with the command:

In [ ]:
docker-machine ls

In [ ]:
To connect to a server you can use:

In [ ]:
docker-machine ssh <machine_name>

In [ ]:
Docker CLI always uses the API. So you can configure the CLI to use a remote host instead of your local Unix socket. That way your client will act as usual but instead of managing your local engine, it will manage a remote one.
Example, suppose you want to interact with the dockerm1 machine. Just type the following command:

In [ ]:
docker-machine env dockerm1

In [ ]:
The above command will provide the env variable and the command to export them in the environment. So using

In [ ]:
eval $(docker-machine env dockerm1)

In [ ]:
you can now work with Docker as usual, however all commands passed will operate on the remote host.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#551199;"></i>&nbsp;&nbsp;Next Steps

# Conclusion

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="4-WKSHP-Conclusion.ipynbb" target="New" title="Next : Conclusion"><i class="fas fa-chevron-circle-right" style="color:#551199;"></i></a></h2>

</br>
 <a href="2-WKSHP-Using-Docker.ipynb" target="New" title="Back: Using Docker"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="3-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next:Conclusion"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
